In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Methods of Projection

The fact that both a line and plane share a vector representation, and may be represented by the pairing of a Decod.es Point and Vec, means that many common operations exhibit similar mathematics.


### Projection Onto Lines

We saw how the vector dot product enables the projection of a vector onto another vector. Through this same method, the projection of Points onto Segments can be achieved.

Given a LinearEntity $L$ that lies in the direction of vector $\vec{v}$, here we find the point $P_{near}$ that lies at a location along $L$ nearest to another point $P$. 

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.06.P05.jpg" style="width: 200px; display: inline;">

First, we may observe that the vector connecting $P$ to $P_{near}$ is perpendicular to the line vector $\vec{v}$. Without relying upon the `vec.projected()` method, we first form the vector connecting $P$ to some known point $P_{0}$ on the line (the start point, for example) and project this in the direction of the line. This can be expressed as:

\begin{align}
\vec{p_{near}} = p_{0} + (( \vec{p} - \vec{p_{0}} ) \bullet \frac{\vec{v}}{|\vec{v}|})\frac{\vec{v}}{|\vec{v}|}
\end{align}

This formulation has the advantage that the parameter value $t$, which identifies how far along $L$ we may find $P_{near}$, can be readily extracted.

\begin{align}
t = ((\vec{p} - \vec{p_{0}})\bullet\frac{\vec{v}}{|\vec{v}|})\frac{1}{|\vec{v}|} = \frac{( \vec{p} - \vec{p_{0}} ) \bullet \vec{v}}{\vec{v}\bullet\vec{v}}
\end{align}

Having found $P_{near}$, the distance between $P$ and $L$ may be determined by $P - P_{near}$. These quantities are all captured by the `line.near()` method, which returns not only the nearest Point, but all the relevant information produced by the calculations, packaged together as a Tuple of three values.

In [ ]:
"""
Line Projection
To project a Point to a Line, we take advantage of the utility of the dot 
product for calculating vector projections. Here, the variable t describes the 
distance along the given Line, described as a percentage of the Line's vector, 
at which the projection of proj_pt will fall. Three values are returned as a 
tuple: the projected Point, the parameter t, and the distance of projection.
"""
def near(self, proj_pt):
    t = Vec(self.pt, proj_pt).dot(self.vec)/vec.length2
    near_pt = self.eval(t)
    return (near_pt, t, proj_pt.dist(near_pt))

For finite LinearEntities, an additional check is required to determine that the result of the projection falls within the valid bounds, and to return the appropriate termination Point otherwise. 

For this we rely on inheritance, first converting the LinearEntity to a Line, and acting according to the results. 

A Ray must check for positive values for the parameter `t` , while Segments must check for values within the range of `0` to `1`.

In [ ]:
"""
Ray Projection
To project a Point to a Ray, we first perform a projection onto an analogous 
Line, and then modify the result if the projected Point falls before the start 
of the directional Ray.
"""
def near(self, proj_pt):
    line_result = self.to_line().near(proj_pt)
    if line_result[1] < 0: return (self.spt, 0.0, proj_pt.dist(self.spt))
    return line_result

In [ ]:
"""
Segment Projection
To project a Point to a Segment, we first perform a projection onto an analogous
Line, and then modify the result if the projected Point falls outside the bounds 
of the Segment.
"""    
def near(self, proj_pt):
    line_result = self.to_line().near(proj_pt)
    if line_result[1] < 0: return (self.spt, 0.0, proj_pt.dist(self.spt))
    if line_result[1] > 1: return (self.ept, 1.0, proj_pt.dist(self.ept))
    return line_result

### Projection Onto Planes

In a manner very similar to Lines, projection of a given point $P$ to a plane may be found by first forming a vector to a point $P_{0}$ that lies on the plane, and then projecting onto a line that starts at this point and lies in the direction of the normal of the plane.

Since the dot product $(\vec{p} - \vec{p_{0}}) \bullet \vec{n}$ is a signed distance, positive when $P$ is on the same side of the plane as the normal is pointing and negative otherwise, the nearest point can then be expressed as:

\begin{align}
\vec{p_{near}} = \vec{p} - ((\vec{p} - \vec{p_{0}})\bullet \vec{n} ) \vec{n}
\end{align}

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.06.P12.jpg" style="width: 200px; display: inline;">

Formulated as a method of the Decod.es Plane class, we can see the commonalities shared with the projection of Points onto Lines.

In [ ]:
"""
Plane Projection
The projection of a Point onto a Plane proceeds along the Plane normal. To 
calculate the distance of projection along this trajectory, we first construct 
a Line from the Plane origin in the direction of the normal. The projection of 
the given Point onto this Line yields a parameter that describes the projection 
distance.
"""
def near(self, proj_pt):
    t = Line(self.origin, self.normal).near(proj_pt)[1]
    tvec = -self.normal*t
    return (proj_pt + tvec, t, tvec.length)

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.06.P13.jpg" style="width: 200px; display: inline;">

One additional term. 

The angle of incidence (AOI) between a ray and a plane is the angle between the ray and the normal of the plane. This quantity effectively measures how directly the ray strikes the surface. For convenience, often the cosine of this angle is calculated, which reduces to a calculation of dot products.